# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [7]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "deepseek-r1"

In [8]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [9]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [11]:
# Let's just make sure the model is loaded

!ollama pull deepseek-r1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling e6a7edc1a4d7: 100% ▕██████████████████▏ 5.2 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling ed8474dc73db: 100% ▕██████████████████▏  179 B                         
pulling f64cd5418e4b: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [12]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

<think>
Okay, user wants to know about business applications of Generative AI. That's a pretty broad question but also super timely since gen AI is exploding right now. 

Hmm, first I should acknowledge that this is an area where people are really excited but sometimes overhyped too. The user probably wants practical examples they can relate to or implement at their job. Maybe they're a business professional looking for ways to leverage this tech.

Let me structure this by industries since that's the most common way businesses think about applications. I should cover both obvious uses like content creation and more niche ones like code generation because those might surprise people but are actually powerful.

Wait, is the user technical or non-technical? The question doesn't specify. Better keep explanations simple for non-tech folks while still being precise enough that tech people get value too. 

I remember there's a big debate about whether gen AI should replace human workers or au

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [13]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])


<think>
Okay, user asked about business applications of Generative AI. That's a broad topic but also super relevant right now since GenAI is all the buzz. Let me unpack this systematically.

First, I should consider who might be asking this. Could be a business professional exploring new tech frontiers, maybe an executive looking for investment ideas or an IT manager evaluating tools. The question feels practical - they probably want concrete use cases rather than theoretical potential. 

Hmm, generating a comprehensive list requires mapping both common and emerging applications across different departments. I'll start with the most impactful areas like marketing since that's often where businesses first encounter GenAI through chatbots etc.

For each application area, I should explain: what it is, why companies care (pain points/business goals), how it works technically at a high level but keep explanations simple enough for non-technical readers. The user didn't specify their familia

## Alternative approach - using OpenAI python library to connect to Ollama

In [16]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

<think>
Okay, user wants to know about business applications of generative AI. That's a broad question but also super relevant right now since everyone is talking about ChatGPT and similar tools.

Hmm, they didn't specify if they're looking for examples in general or particular industries, so maybe they just want an overview? I should structure this logically - start with content creation since that's the obvious one, then move to support/operations, design, etc. 

Wait, is their company tech-focused and considering AI adoption? Or are they just researching concepts? Since they didn't give much context, I'll assume they need general knowledge but practical examples.

For content stuff: marketing teams would eat this up - copy generation for social media saves hours. But IT departments might worry about consistency... so maybe mention template systems to maintain brand voice where needed.

Oh and customer service! That's massive. Chatbots can handle common queries now rather than just g

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [14]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 720 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 6.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 9.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  14 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  22 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  26 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   4% ▕                  ▏  43 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   5% ▕                  ▏  51 MB/1.1 GB                  pulling manifest 
pulling aabd4deb

In [17]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I need to define some core concepts related to Large Language Models (LLMs), specifically focusing on neural networks, attention, and transformers. Hmm, let me think about how these components work together.

Starting with neural networks. From my basic understanding, neural networks are a set of interconnected nodes or neurons that process input data. These nodes have weights associated with them, which determine the strength of their influence on others. They learn by adjusting these weights based on trained examples. So, in the context of LLMs, these neural networks might help in recognizing patterns and generating text.

Wait, so if I'm not mistaken, each LLM's core is a neural network that processes input and predicts outputs, right? But I'm a bit fuzzy on the exact structure—like how many layers or nodes there are. Oh, yeah, RNNs (Recurrent Neural Networks) might be used because they handle sequential data, which text generation needs.

Now moving to attention. F

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [38]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [27]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [33]:

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [34]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [35]:

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [44]:

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [46]:

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [47]:
display_summary("https://www.cnn.com/")

Here's a summary of the news and current events:

**Politics**

* President Trump has been critical of the Federal Reserve, calling its Chair Powell an "average mentally person" and hinting that he is working on replacements.
* The Supreme Court blessed same-sex marriage 10 years ago. However, some observers say there may be a backlash brewing.

**Business**

* Shell denies reports it is in talks to take over BP.
* Buying now, pay later loans will factor into Americans' credit scores, according to a new study.

**Sports**

* Aaron Rodgers has said he thinks this upcoming season is his last as quarterback for the Pittsburgh Steelers.
* The NBA Draft is scheduled to begin next week.

**Entertainment**

* Steve Burns, the original host of "Blue's Clues," is launching a podcast for grown-ups.
* Taylor Swift surprised fans at Tight End University as she supported friend Travis Kelce.

**World**

* Venezuelan government influencer who criticized gangs and police was shot dead during a TikTok livestream.
* Brazilian tourist found dead on ridge of Indonesian volcano.
* Ukraine-Russia War: The conflict continues to escalate in Eastern Europe.

**Health**

* Microplastics shed from packaging are contaminating food and drink, according to a new study.
* Climate Change: Rising temperatures threaten global food production.
* NASA is exploring the potential of biodesign in cities of the future, developing building materials that can be grown with properties like self-coloring and healing.

**Other News**

* Elon Musk has announced plans to send humans to Mars in the coming years.
* The Queen Elizabeth II Memorial design has been unveiled.
* New tariffs set by President Trump could affect global trade.